In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import geocoder

In [2]:
# URL imported via requests library and processed with BeautifulSoup
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_text = requests.get(url).text
soup = BeautifulSoup(html_text, 'html.parser')

In [3]:
# Below code finds the td elements in the table
neighTable = soup.find('table')
nt = neighTable.findAll('td')

In [4]:
# For loop used to get only the text from the td and append it to a new list
nt_text = []
for sa in nt:
    test = sa.get_text()
    nt_text.append(test)

In [5]:
len(nt_text)

540

In [6]:
# 3 different lists to hold the 3 column values. For loop uses index to specify which column the value belongs to
Postal_Code = []
Borough = []
Neighborhood = []
n = 0
while n < len(nt_text):
    Postal_Code.append(nt_text[n])
    Borough.append(nt_text[n+1])
    Neighborhood.append(nt_text[n+2])
    n+=3   

In [7]:
# 3 lists added to a dictionary and converted to a dataframe
Dictionary = {'PostalCode':Postal_Code, 'Borough':Borough, 'Neighborhood':Neighborhood}
Canada = pd.DataFrame(Dictionary)
Canada.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [8]:
# Used RegEx to remove the Newline values
Canada['PostalCode'] = Canada.PostalCode.str.extract(r'(.*)\n')
Canada['Borough'] = Canada.Borough.str.extract(r'(.*)\n')
Canada['Neighborhood'] = Canada.Neighborhood.str.extract(r'(.*)\n')
Canada.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
# 77 Not assigned values that need to be removed from Borough
Canada.Borough.value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [10]:
# Created a new dataframe that has the Not assigned values removed from the dataframe
Canada = Canada[Canada['Borough'] != 'Not assigned']
Canada.Borough.value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [11]:
# Checked Neighborhood column and found no Not assigned values
Canada[Canada['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood


In [12]:
Canada.shape

(103, 3)

### Adding Latitude and Longitude

In [13]:
# Import Latitude and Longitude data and rename Postal Code column
latlon = pd.read_csv("https://cocl.us/Geospatial_data")
latlon.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
latlon.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
# Joined new data onto our original dataframe
CAN = pd.merge(Canada, latlon, on='PostalCode', how='left')
CAN.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [15]:
# Amount of rows stayed the same, 2 extra columns added
CAN.shape

(103, 5)

### Explore and Cluster

In [16]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [17]:
CAN['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [18]:
# Work with the Boroughs that only contain Toronto. Use RegEx to match any value containing Toronto
Toronto = CAN.copy()
Toronto['Borough'] = Toronto['Borough'].str.extract(r'.*?(.*?Toronto).*?')
# 39 neighborhoods in Toronto boroughs
Toronto['Borough'].value_counts().sum()

39

In [19]:
# Filling the Null values to be removed.
Toronto['Borough'].fillna('0', inplace=True)
Toronto['Borough'].value_counts()

0                   64
Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [20]:
# Filter out the 0 for borough
TBorough = Toronto[Toronto['Borough'] != '0'].reset_index(drop=True)
TBorough['Borough'].value_counts()

Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [21]:
TBorough.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [22]:
# Pull the Latitude and Longitude Values of Toronto
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [23]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood, postal_c in zip(TBorough['Latitude'], TBorough['Longitude'], TBorough['Borough'], TBorough['Neighborhood'], TBorough['PostalCode']):
    label = '{}, {}, {}'.format(postal_c, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [24]:
# set number of clusters
kclusters = 4

toronto_clustering = TBorough.drop(['Neighborhood','PostalCode','Borough'], axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# Cluster labels for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 0, 3, 3, 1, 3, 1])

In [25]:
# add clustering labels to dataframe
TBorough.insert(0, 'Cluster Labels', kmeans.labels_)
TBorough.head()

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,3,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,3,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,0,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(TBorough['Latitude'], TBorough['Longitude'], TBorough['Neighborhood'], TBorough['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters